In [2]:
import polars as pl
import requests
from pathlib import Path

from src.file_handling import file_location
from src.corpus_parsing import asme_digitial_jmd

folder_path = file_location.FileLocation()
root_path = folder_path.root
data_path = root_path.parent.parent / 'fyp' / 'data'

asme_path = file_location.FolderPathOfASME(data_path)
asme_html_issues_path = asme_path.asme_jmd_html_issues
jmd_bare_paper_infor_path = asme_path.asme_jmd_pdf / 'jmd_papers_bare_infor.parquet'

In [2]:
if not jmd_bare_paper_infor_path.exists():
    all_issues_infor_df :pl.DataFrame = asme_digitial_jmd.all_issues_in_folder_to_df(asme_html_issues_path)

    def cluster_url_by_issue_remotexs(
            issues_dfs: pl.DataFrame
    )-> pl.DataFrame:
        df :pl.DataFrame = issues_dfs.select('year','volume','issue').unique(maintain_order=True)
        df = df.join(all_issues_infor_df, on=['year','volume','issue'])
        df = df.with_columns(
            (pl.col('pdf_url').str.split('.org').list.get(0) +
            pl.lit('.org') + pl.lit('.remotexs.ntu.edu.sg') +
             pl.col('pdf_url').str.split('.org').list.get(1)
             ).alias('pdf_url_remotexs')
        )
        df = df.with_columns(
            (pl.col('article_url').str.split('.org').list.get(0) +
            pl.lit('.org') + pl.lit('.remotexs.ntu.edu.sg') +
             pl.col('article_url').str.split('.org').list.get(1)
             ).alias('article_url_remotexs')
        )
        return df

    cluster_url_by_issue_df = cluster_url_by_issue_remotexs(issues_dfs=all_issues_infor_df)
    cluster_url_by_issue_df.write_parquet(jmd_bare_paper_infor_path)
else:
    cluster_url_by_issue_df = pl.read_parquet(jmd_bare_paper_infor_path)

cluster_url_by_issue_df

year,volume,issue,title,authors,doi,month,article_url,pdf_url,topics,doi_filename,pdf_url_remotexs,article_url_remotexs
str,str,str,str,str,str,str,str,str,str,str,str,str
"""1978""","""100""","""1""","""Editorial""","""Charles W. McLarnan""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""""","""doi_10.1115_1.3453887""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Fatigue Life Prediction of Com…","""B. N. Leis""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Fatigue life;Fatigue;Fatigue a…","""doi_10.1115_1.3453889""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Dynamic Severity Criterion for…","""G. S. A. Shawki""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Design;High cycle fatigue;Stre…","""doi_10.1115_1.3453874""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Wear-Related Topography of Ion…","""D. J. Sturges;S. W. Martin;C. …","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Wear;Nitriding;Composite mater…","""doi_10.1115_1.3453882""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Simulation of Resonances and I…","""M. Benton;A. Seireg""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Gears;Resonance;Simulation;Com…","""doi_10.1115_1.3453888""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2026""","""148""","""3""","""Product Dataset Platform: Syst…","""Mohammad Arjomandi Rad;Julian …","""https://doi.org/10.1115/1.4068…","""March""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Design;Sleep""","""doi_10.1115_1.4068755""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""2026""","""148""","""3""","""Concept Chaining Patterns Duri…","""Madhurima Das;Jessica Meza;Chr…","""https://doi.org/10.1115/1.4068…","""March""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Chain;Design""","""doi_10.1115_1.4068983""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""2026""","""148""","""3""","""A Quasi-Optimal Shape Design M…","""Sifan Chen;Guoyue Luo;Yuan Kon…","""https://doi.org/10.1115/1.4068…","""March""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Construction;Cutting;Optimizat…","""doi_10.1115_1.4068955""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"


### request blocked by bot Detection

In [ ]:
r = requests.get('https://asmedigitalcollection.asme.org.remotexs.ntu.edu.sg/mechanicaldesign/article-pdf/doi/10.1115/1.2918913/5923895/071402_1.pdf')
r

In [9]:
clustered_url_gte_2022_df = cluster_url_by_issue_df.filter(pl.col('year')>='2021')

url_to_open_list = clustered_url_gte_2022_df['pdf_url_remotexs'].to_list()
doi_filename_list = clustered_url_gte_2022_df['doi_filename'].to_list()
pdf_folder :Path = asme_path.asme_jmd_pdf
if not pdf_folder.exists():
    pdf_folder.mkdir()
pdf_save_paths_list = [pdf_folder / (doi_filename + '.pdf') for doi_filename in doi_filename_list]
test_pdf_url = url_to_open_list[0]
pdf_save_paths_list

[WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047001.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047434.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047345.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047346.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047437.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047436.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological University/Academics/fyp/data/asme_jmd/pdf/doi_10.1115_1.4047435.pdf'),
 WindowsPath('F:/One Drives/OneDrive - Nanyang Technological Universi

### initial webdriver cookie setup

### save pdf to data folder

In [ ]:
import undetected_chromedriver as webdriver

# login to remotexs
driver = webdriver.Chrome()
driver.get(test_pdf_url)

input('Enter anything in cli to continue...')


for save_path, pdf_url in zip(pdf_save_paths_list, url_to_open_list):
    if save_path.exists():
        continue
    driver.execute_script(f"window.open('{pdf_url}', '_blank');")
    driver.switch_to.window(driver.window_handles[-1])

    # Download original PDF with requests (session cookies preserved)
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    r = requests.get(driver.current_url, cookies=cookies)
    save_path.write_bytes(r.content)

    driver.close()
    driver.switch_to.window(driver.window_handles[0])


### save html to data folder

In [ ]:
url_to_open_list = cluster_url_by_issue_df['article_url_remotexs'].to_list()
doi_filename_list = cluster_url_by_issue_df['doi_filename'].to_list()
article_html_folder :Path = asme_path.article_html

if not article_html_folder.exists(): article_html_folder.mkdir()

html_folder_paths_list = [article_html_folder / (doi_filename + '.html') for doi_filename in doi_filename_list]
test_pdf_url = url_to_open_list[0]
html_folder_paths_list

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec

# login to remotexs
driver = webdriver.Chrome()
driver.get(test_pdf_url)

input('Press enter to continue...')

for save_path, pdf_url in zip(html_folder_paths_list, url_to_open_list):
    if save_path.exists():
        continue
    driver.execute_script(f"window.open('{pdf_url}', '_blank');")
    driver.switch_to.window(driver.window_handles[-1])
    selector = "#Sidebar > div.sidebar-widget_wrap > div > div > div > div > div > div > div.widget-DynamicWidgetLayout.widget-instance-Article_RightRailB0B6 > div > div > div > div > h3"
    WebDriverWait(driver, 15).until(
        ec.presence_of_element_located((By.CSS_SELECTOR, selector))
    )

    save_path.write_text(driver.page_source, encoding="utf-8")

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [12]:
from bs4 import BeautifulSoup
from datetime import datetime as dt
test_html_path :Path = list(article_html_folder.rglob('*.html'))[-1]
soup_article = BeautifulSoup(test_html_path.read_text(encoding="utf-8"), "lxml")

In [13]:
test_pdf_urlclient_type :str = soup_article.select('span.article-client_type')[0].text.strip()
title_paper :str = soup_article.select('title')[0].text.strip().split('|')[0].strip()
author_full_names :str = ';'.join([name_html.text.strip() for name_html in soup_article.select('div.author-full-name')])
author_affiliation :str = ';'.join([aff.text.strip() for aff in soup_article.select('div.author-affiliation > div.aff')])
author_emails :str = ';'.join([email_html.text.strip() for email_html in soup_article.select('div.article-footnote > a[href]')])
publish_date = dt.strptime(soup_article.select('span.publish-date-label')[0].text.strip().split(':')[1].strip(),'%B %d, %Y')
abstract :str = soup_article.select('div.article-section-wrapper > section.abstract > p')[0].text.strip()
issue_section :str = soup_article.select('div.content-metadata-tocSections > a[href]')[0].text.strip()
keywords = ';'.join([keyword_html.text.strip() for keyword_html in soup_article.select('div.content-metadata-keywords > a[href]')])
topics = ';'.join([topic_html.text.strip() for topic_html in soup_article.select('div.content-metadata-topics > a[href]')])
content_sections :list = [section_html.text.strip() for section_html in soup_article.select('h2.section-title')]
paragraphs_sections :list = []
for section_paragraph in soup_article.select('h2.section-title + div'):
    _section :list = []
    for paragraph in section_paragraph.select('div > div > p'):
        _section += [paragraph.text.strip()]
    paragraphs_sections.append('\n'.join(_section))
article_content :list = [f'{header}:{paragraphs}' for header, paragraphs in zip(content_sections, paragraphs_sections)]

acknowledgement :str = soup_article.select('h2.backacknowledgements-title + div')[0].text.strip()
back_section_header :list = [header_html.text.strip() for header_html in soup_article.select('h2.backsection-title')]
back_section_content :list = [content_html.text.strip() for content_html in soup_article.select('h2.backsection-title + div > div > p')]
article_back_section :dict = {header:content for header,content in zip(back_section_header, back_section_content)}

cited_article_columns :tuple = ('title', 'cited_authors','doi_links','source')
cited_article_infor :dict = {col:[] for col in cited_article_columns}
for ref in soup_article.select("div.ref-list")[0].select('div.citation'):
    ref_title, ref_authors, ref_doi, ref_source = None, None, None, None
    if ref.select('div.article-title'): ref_title = ref.select('div.article-title')[0].text.strip()
    if ref.select('span.person-group'): ref_authors = ref.select('span.person-group')[0].text.strip()
    if ref.select('div.crossref-doi'): ref_doi = ref.select('div.crossref-doi a')[0]['href'].replace('dx.', '')
    if ref.select('div.source'): ref_source = ref.select('div.source')[0].text.strip()
    cited_article_infor['title'] += [ref_title]
    cited_article_infor['cited_authors'] += [ref_authors]
    cited_article_infor['doi_links'] += [ref_doi]
    cited_article_infor['source'] += [ref_source]

pl.DataFrame(cited_article_infor)


IndexError: list index out of range

### check prior 2021 data

In [3]:
if not jmd_bare_paper_infor_path.exists():
    all_issues_infor_df :pl.DataFrame = asme_digitial_jmd.all_issues_in_folder_to_df(asme_html_issues_path)

    def cluster_url_by_issue_remotexs(
            issues_dfs: pl.DataFrame
    )-> pl.DataFrame:
        df :pl.DataFrame = issues_dfs.select('year','volume','issue').unique(maintain_order=True)
        df = df.join(all_issues_infor_df, on=['year','volume','issue'])
        df = df.with_columns(
            (pl.col('pdf_url').str.split('.org').list.get(0) +
            pl.lit('.org') + pl.lit('.remotexs.ntu.edu.sg') +
             pl.col('pdf_url').str.split('.org').list.get(1)
             ).alias('pdf_url_remotexs')
        )
        df = df.with_columns(
            (pl.col('article_url').str.split('.org').list.get(0) +
            pl.lit('.org') + pl.lit('.remotexs.ntu.edu.sg') +
             pl.col('article_url').str.split('.org').list.get(1)
             ).alias('article_url_remotexs')
        )
        return df

    cluster_url_by_issue_df = cluster_url_by_issue_remotexs(issues_dfs=all_issues_infor_df)
    cluster_url_by_issue_df.write_parquet(jmd_bare_paper_infor_path)
else:
    cluster_url_by_issue_df = pl.read_parquet(jmd_bare_paper_infor_path)

cluster_url_by_issue_df

year,volume,issue,title,authors,doi,month,article_url,pdf_url,topics,doi_filename,pdf_url_remotexs,article_url_remotexs
str,str,str,str,str,str,str,str,str,str,str,str,str
"""1978""","""100""","""1""","""Editorial""","""Charles W. McLarnan""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""""","""doi_10.1115_1.3453887""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Fatigue Life Prediction of Com…","""B. N. Leis""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Fatigue life;Fatigue;Fatigue a…","""doi_10.1115_1.3453889""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Dynamic Severity Criterion for…","""G. S. A. Shawki""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Design;High cycle fatigue;Stre…","""doi_10.1115_1.3453874""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Wear-Related Topography of Ion…","""D. J. Sturges;S. W. Martin;C. …","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Wear;Nitriding;Composite mater…","""doi_10.1115_1.3453882""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""1978""","""100""","""1""","""Simulation of Resonances and I…","""M. Benton;A. Seireg""","""https://doi.org/10.1115/1.3453…","""January""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Gears;Resonance;Simulation;Com…","""doi_10.1115_1.3453888""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2026""","""148""","""3""","""Product Dataset Platform: Syst…","""Mohammad Arjomandi Rad;Julian …","""https://doi.org/10.1115/1.4068…","""March""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Design;Sleep""","""doi_10.1115_1.4068755""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""2026""","""148""","""3""","""Concept Chaining Patterns Duri…","""Madhurima Das;Jessica Meza;Chr…","""https://doi.org/10.1115/1.4068…","""March""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Chain;Design""","""doi_10.1115_1.4068983""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"
"""2026""","""148""","""3""","""A Quasi-Optimal Shape Design M…","""Sifan Chen;Guoyue Luo;Yuan Kon…","""https://doi.org/10.1115/1.4068…","""March""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…","""Construction;Cutting;Optimizat…","""doi_10.1115_1.4068955""","""https://asmedigitalcollection.…","""https://asmedigitalcollection.…"


In [23]:
doi_pdf_filenames :list = [doi_filename + '.pdf' for doi_filename in cluster_url_by_issue_df['doi_filename'].to_list()]
available_doi_pdf_list = list(asme_path.asme_jmd_pdf.rglob('*.pdf'))
available_doi_list :list = [doi_pdf_filename.name.split('.pdf')[0] for doi_pdf_filename in available_doi_pdf_list]
pdf_not_available_df = cluster_url_by_issue_df.filter(~pl.col('doi_filename').is_in(available_doi_list))


url_to_open_list = pdf_not_available_df['pdf_url_remotexs'].to_list()
doi_filename_list = pdf_not_available_df['doi_filename'].to_list()
pdf_folder :Path = asme_path.asme_jmd_pdf
if not pdf_folder.exists():
    pdf_folder.mkdir()
pdf_save_paths_list = [pdf_folder / (doi_filename + '.pdf') for doi_filename in doi_filename_list]
test_pdf_url = url_to_open_list[0]

import undetected_chromedriver as webdriver

# login to remotexs
driver = webdriver.Chrome()
driver.get(test_pdf_url)

input('Enter anything in cli to continue...')


for save_path, pdf_url in zip(pdf_save_paths_list, url_to_open_list):
    if save_path.exists():
        continue
    driver.get(pdf_url)

    # Download original PDF with requests (session cookies preserved)
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    r = requests.get(driver.current_url, cookies=cookies)
    save_path.write_bytes(r.content)

    driver.switch_to.window(driver.window_handles[0])
